In [2]:
from keras.models import load_model

In [3]:
model = load_model('./models/baseline_model.h5')

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 128)      

In [15]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [8]:
batch_size = 1000

In [26]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(32, 32),
    batch_size=batch_size,
    shuffle=False
)

Found 300000 images belonging to 1 classes.


In [27]:
predictions = model.predict_generator(test_generator,
                                      steps=test_generator.n // batch_size,
                                      workers=8,
                                      verbose=1)

300/300 [==============================] - 134s 448ms/step


In [28]:
predictions.shape

(300000, 10)

In [29]:
predict_classes = np.argmax(predictions, axis=1)
print(predict_classes.shape)

(300000,)


In [30]:
predict_classes[:10]

array([4, 3, 4, 4, 4, 4, 4, 4, 6, 1])

In [52]:
import os
classes = sorted(os.listdir('./data/train'))
idx2name = {}
for i, x in enumerate(classes):
    idx2name[i] = x
idx2name

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [32]:
predict_classes = [idx2name[x] for x in predict_classes]
predict_classes[:10]

['deer',
 'cat',
 'deer',
 'deer',
 'deer',
 'deer',
 'deer',
 'deer',
 'frog',
 'automobile']

In [61]:
# flow_from_directory()は文字列でソートしてしまうのでファイル名の数値でソートして結果を並び替える
filenames = test_generator.filenames
filenames[:10]

['unknown/1.png',
 'unknown/10.png',
 'unknown/100.png',
 'unknown/1000.png',
 'unknown/10000.png',
 'unknown/100000.png',
 'unknown/100001.png',
 'unknown/100002.png',
 'unknown/100003.png',
 'unknown/100004.png']

In [62]:
file_numbers = [int(f.replace('unknown/', '').replace('.png', '')) for f in filenames]
file_numbers[:10]

[1, 10, 100, 1000, 10000, 100000, 100001, 100002, 100003, 100004]

In [63]:
sorted_idx = np.argsort(file_numbers)
sorted_idx[:10]

array([     0, 111111, 222222, 233334, 244445, 255556, 266667, 277778,
       288889,      1])

In [67]:
predict_classes = np.array(predict_classes)
sorted_predict_classes = predict_classes[sorted_idx]

In [68]:
submissions = pd.DataFrame({
    'id': list(range(1, len(predict_classes) + 1)),
    'label': sorted_predict_classes
})

In [69]:
submissions

,id,label
0,1,deer
1,2,airplane
2,3,automobile
3,4,ship
4,5,bird
...,...,...
299995,299996,cat
299996,299997,deer
299997,299998,deer
299998,299999,frog


In [70]:
submissions.to_csv('submits/baseline_model.csv', index=False, header=True)